# Ollama

## Librerías

In [1]:
!pip install ollama

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser

/home/jggomez/Desktop/IRIS/iris-uc3m/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prompt

In [3]:
# 1. Tu diccionario original
label_dict = {
    0: "Alegría",
    1: "Miedo",
    2: "Sorpresa",
    3: "Tranquilidad",
    4: "Tristeza",
    5: "Adversión",
    6: "Gratitud",
    7: "Ira"
}

# Convertimos el dict a string para inyectarlo en el prompt claramente
labels_str = "\n".join([f"- {k}: {v}" for k, v in label_dict.items()])

# 2. El Template Adaptado
template = """
Eres un experto lingüista especializado en análisis de emociones y psicología.

TU TAREA:
Analizar la frase proporcionada y clasificarla en UNA de las categorías emocionales permitidas.

CATEGORÍAS PERMITIDAS (ID: Emoción):
{labels_list}

INSTRUCCIONES DE SALIDA:
Debes responder ÚNICAMENTE con un objeto JSON válido. No incluyas texto, markdown o explicaciones fuera del JSON.

FORMATO JSON ESPERADO:
{{
    "emocion": "Nombre de la emoción (string)",
    "id_etiqueta": ID numérico (int),
    "evidencia": "Palabras o fragmentos clave del texto que justifican la decisión",
    "razonamiento": "Breve explicación de por qué encaja en esta categoría"
}}

PASOS PARA RAZONAR:
1. Lee el texto e identifica la carga emocional predominante.
2. Descarta las emociones que no encajen.
3. Selecciona la etiqueta (ID y Nombre) que mejor describa la intención del texto.
4. Extrae la evidencia literal.
5. Genera el JSON.

TEXTO A ANALIZAR:
"{texto}"
"""

## Modelo

Se pueden usar otros modelos cambiando el `model`.

*Importante*: los modelos se tienen que haber descargado antes en la terminal usando `ollama pull "nombre_modelo"`

In [4]:
# 3. Configuración del Prompt y Modelo
prompt = ChatPromptTemplate.from_template(template)
llm = ChatOllama(model="gemma3:4b", temperature=0.0, format="json") 
# Nota: temperature=0.0 es mejor para clasificación estricta. format="json" fuerza a Ollama a ser estructurado.

chain = prompt | llm | JsonOutputParser()

/tmp/ipykernel_1002483/3663837849.py:3: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="gemma3:4b", temperature=0.0, format="json")


## Ejecutar

In [5]:
# 4. Ejecución
texto_ejemplo = "No puedo creer lo que han hecho, es absolutamente indignante y me hierve la sangre ver tanta injusticia."

res = chain.invoke({
    "labels_list": labels_str,
    "texto": texto_ejemplo
})

print(res)

{'emocion': 'Ira', 'id_etiqueta': 7, 'evidencia': 'es absolutamente indignante y me hierve la sangre', 'razonamiento': "La frase expresa una fuerte reacción de enojo y rechazo ante la injusticia, utilizando lenguaje figurado ('me hierve la sangre') que indica una intensa emoción negativa."}
